#### Import libraries

In [18]:
from pyproj import CRS
import geopandas as gpd

# Import libraries
import pandas as pd
import geopandas as gpd
import re
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import contextily as ctx
from pyproj import CRS
from geopandas.tools import geocode
import requests
import geojson
import mplleaflet
import folium

#### Function to read population shape files

In [45]:
def read_population_data(year):
    
    # Define path to file
    fp = "population_data/"+str(year)+"/Vaestotietoruudukko_"+str(year)+".shp"
    
    # Read shapefile
    pop = gpd.read_file(fp)
    
    # Define coordinate reference system
    pop.crs = CRS.from_epsg(3879).to_wkt()
    
    # Drop columns if number of resudents is less than 0 (no value) or over 10000 (values without real location)
    pop = pop[(pop["ASUKKAITA"] > 0) & (pop["ASUKKAITA"] < 10000)]
    
    # Drop all the columns except index and asukkaita
    pop_pop = pop[["INDEX", "ASUKKAITA"]]
    pop_geom = pop[["INDEX", "geometry"]]
    
    # Print number of residents by year
    print(year, "number of residents:", pop_pop["ASUKKAITA"].sum())
    
    # Rename column asukkaita to pop+year
    pop_pop = pop_pop.rename(columns={'ASUKKAITA': 'pop'+str(year)})
    
    # Rename column geometry to geometry+year. Must do to later fill missing values and merge tables.
    pop_geom = pop_geom.rename(columns={'geometry': 'geometry'+str(year)})
    
    # Set column INDEX to INDEX
    pop_pop = pop_pop.set_index("INDEX")
    pop_geom = pop_geom.set_index("INDEX")
    
    # Return result geodataframe
    return pop_pop, pop_geom

#### Read the files

In [46]:
# Data is available from years 1997-2003 and 2008-2019

# Years 1997 to 2003
#pop1997_pop, pop1997_geom = read_population_data(1997)
#pop1998_pop, pop1998_geom = read_population_data(1998)
#pop1999_pop, pop1999_geom = read_population_data(1999)
#pop2000_pop, pop2000_geom = read_population_data(2000)
#pop2001_pop, pop2001_geom = read_population_data(2001)
#pop2002_pop, pop2002_geom = read_population_data(2002)
#pop2003_pop, pop2003_geom = read_population_data(2003)

# Years 2008 to 2012
#pop2008_pop, pop2008_geom = read_population_data(2008)
#pop2009_pop, pop2009_geom = read_population_data(2009)
#pop2010_pop, pop2010_geom = read_population_data(2010)
#pop2011_pop, pop2011_geom = read_population_data(2011)
#pop2012_pop, pop2012_geom = read_population_data(2012)

# Years 2013 to 2019
pop2013_pop, pop2013_geom = read_population_data(2013)
pop2014_pop, pop2014_geom = read_population_data(2014)
pop2015_pop, pop2015_geom = read_population_data(2015)
pop2016_pop, pop2016_geom = read_population_data(2016)
pop2017_pop, pop2017_geom = read_population_data(2017)
pop2018_pop, pop2018_geom = read_population_data(2018)
pop2019_pop, pop2019_geom = read_population_data(2019)

2013 number of residents: 1049988
2014 number of residents: 1076765
2015 number of residents: 1087308
2016 number of residents: 1111940
2017 number of residents: 1121567
2018 number of residents: 1137053
2019 number of residents: 1155285


## Join tables

#### Result will have 3 dataframes. One for 1997 to 2003, one for 2008 to 2012 and one for 2013 to 2019.

In [47]:
from functools import reduce

# Combine dataframes to lists
pop_list = [pop2013_pop, pop2014_pop, pop2015_pop, pop2016_pop, pop2017_pop, pop2018_pop, pop2019_pop]
geom_list = [pop2013_geom, pop2014_geom, pop2015_geom, pop2016_geom, pop2017_geom, pop2018_geom, pop2019_geom]

# Merge dataframes
pop = reduce(lambda left,right: pd.merge(left,right,on='INDEX', how="outer"), pop_list)
geom = reduce(lambda left,right: pd.merge(left,right,on='INDEX', how="outer"), geom_list)

# Fill missing population values with 0
pop = pop.fillna(0)

# Fill missing geometry value from left
geom["geometry"] = geom.apply(lambda x:  x[x.last_valid_index()], axis=1)
geom = geom["geometry"]

# Merge dataframes pop and geom
population = pop.join(geom)

# Convert dataframe back to geodataframe
population = gpd.GeoDataFrame(population)

population.crs = CRS.from_epsg(3879).to_wkt()

# Show dataframe
population

pop2013  pop2014  pop2015  pop2016  pop2017  pop2018  pop2019  \
INDEX                                                                  
688        5.0      8.0      8.0      7.0      9.0      9.0      0.0   
703        6.0      6.0      6.0      6.0      0.0      5.0      5.0   
710        7.0      8.0      8.0      8.0      8.0      8.0      8.0   
711        7.0      7.0      7.0      7.0      5.0      5.0      5.0   
715       16.0     16.0     19.0     17.0     12.0     11.0     12.0   
...        ...      ...      ...      ...      ...      ...      ...   
22925      0.0      0.0      0.0      0.0      0.0      0.0      7.0   
23032      0.0      0.0      0.0      0.0      0.0      0.0      7.0   
23355      0.0      0.0      0.0      0.0      0.0      0.0     21.0   
23866      0.0      0.0      0.0      0.0      0.0      0.0      5.0   
26475      0.0      0.0      0.0      0.0      0.0      0.0      5.0   

                                                geometry  
INDEX                                                     
688    POLYGON Z ((25472499.995 6689749.005 0.000, 25...  
703    POLYGON Z ((25472499.995 6685998.998 0.000, 25...  
710    POLYGON Z ((25472499.995 6684249.004 0.000, 25...  
711    POLYGON Z ((25472499.995 6683999.005 0.000, 25...  
715    POLYGON Z ((25472499.995 6682998.998 0.000, 25...  
...                                                  ...  
22925  POLYGON Z ((25506749.994 6678999.002 0.000, 25...  
23032  POLYGON Z ((25507000.000 6692749.005 0.000, 25...  
23355  POLYGON Z ((25507499.995 6692999.004 0.000, 25...  
23866  POLYGON Z ((25508249.996 6686749.005 0.000, 25...  
26475  POLYGON Z ((25512250.000 6682499.000 0.000, 25...  

[6103 rows x 8 columns]

## Join tables

#### Hox! Between years 1997 and 2012 grid size 500m x 500m. Since 2013 grid size 250m x 250m.

In [53]:
print(population["pop2018"].sum())
print(population["pop2019"].sum())

1137053.0
1155285.0


In [ ]:
# Shop locations

import osmnx as ox
import matplotlib.pyplot as plt

# Plave name
place_name = "Helsinki, Finland"

# Retrieve firestations
#schools = ox.pois_from_place(place_name, amenities=['hospital'])
firestations = ox.pois_from_place(place_name, amenities=['fire_station'])

firestations = firestations[["name", "geometry"]]



In [ ]:
#shops = shops[(shops["amenities"]=="kassa") & (shops["amenities"]=="kassa") & (shops["amenities"]=="kassa")]


#amenities=[["kassa", "shopping_carts", "shopping_cart"]]
# Drop all the point of interest, except school
#kassa, shopping_carts, shopping_cart

# How many restaurants do we have?
#shops["name"]
#schools["name"].unique()

#pois["amenity"].value_counts().head(60)

firestations

In [ ]:
# Print 2012 population 

#2012 Asukkaita: 1036528
print("Population 2008:", pop_until_2012["pop2008"].sum())
print("Population 2012:", pop_until_2012["pop2012"].sum())

pop_until_2012
# Extract old grid system
#old_grid = pop_before_2013[["INDEX", "geometry"]]

# Rename old index column
#old_grid = old_grid.rename(columns={'INDEX': 'INDEX_OLD'})

# Join convert new grid system to old one
#join = gpd.sjoin(pop2013, old_grid, how="inner", op="within")

# Drop duplicates
#join = join[~join.duplicated(subset=['INDEX_OLD','pop2013'], keep=False)]

# Print result
#print("Population 2013:", join["pop2013"].sum())

#print(join)

#### Visualize results

In [51]:
#pop = pop_before_2013.to_crs(epsg=3857)
pop = population.to_crs(epsg=3857)

# Initialize plot
fig, ax = plt.subplots(figsize=(10,10))

# Visualize the areas
#data.plot(ax=ax, column="time_avg", cmap="viridis", linewidth=0, legend=False, alpha=0.9)
#pop.plot(ax=ax, color='green', linewidth=0, legend=False, alpha=0.9)
pop.plot(ax=ax, column="pop2018", cmap="viridis", linewidth=0.5, legend=True, alpha=0.9)


# Add titles to figures
#ax.set_title("Areas where you can reach both Kumpula and Pitäjämäki in less than " + str(time_max) + " minutes.")

# Use tight layout in plot
plt.tight_layout()

# Plot the addresses
#pop2019.plot(ax=ax, color="red", markersize=100)

# Add basemap with `ST_TONER` style
ctx.add_basemap(ax, url=ctx.tile_providers.OSM_A)

# Define output filename for the figure in here and use it when saving the file:
#output_fig = "docs/travel_times.png"

# Save figure with given name
#plt.savefig(output_fig)